In [1]:
import gymnasium as gym


from stable_baselines3 import PPO, A2C
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor
from gymnasium.utils.env_checker import check_env


import gym_anytrading

#env_id = "CartPole-v1"
env_id = "future-v0"

env = gym.make(env_id, frame_bound=(500, 1000), window_size = 300)
check_env(env)

shape is 2402


/Users/liuzhe/miniconda3/envs/torch_env/lib/python3.9/site-packages/gymnasium/utils/env_checker.py:277: UserWarning: WARN: The environment (<OrderEnforcing<PassiveEnvChecker<FutureEnv<future-v0>>>>) is different from the unwrapped version (<FutureEnv<future-v0>>). This could effect the environment checker as the environment most likely has a wrapper applied to it. We recommend using the raw environment for `check_env` using `env.unwrapped`.
  logger.warn(
/Users/liuzhe/miniconda3/envs/torch_env/lib/python3.9/site-packages/gymnasium/utils/env_checker.py:221: UserWarning: WARN: A Box observation space minimum value is -infinity. This is probably too low.
  logger.warn(
/Users/liuzhe/miniconda3/envs/torch_env/lib/python3.9/site-packages/gymnasium/utils/env_checker.py:225: UserWarning: WARN: A Box observation space maximum value is -infinity. This is probably too high.
  logger.warn(
/Users/liuzhe/miniconda3/envs/torch_env/lib/python3.9/site-packages/gymnasium/utils/env_checker.py:186: Use

In [2]:
env.df.head()
#import pandas
#df = pandas.read_parquet("../gym_anytrading/datasets/data/RB_TICK.tick")
#df.columns

,symbol,tradingDay,updateTime,updateMillisec,lastPrice,volume,turnover,askPrice1,askPrice2,askPrice3,...,preDelta,average,diff,av1,av2,av3,bv1,bv2,bv3,pec
636,rb2305,20230307,09:00:00,500,4204.0,339857,1.430642e+10,4204.0,4205.0,4208.0,...,1.797693e+308,42095.407421,NaN,1.716003,1.041393,1.431364,2.671173,2.727541,2.727541,-0.017260
980,rb2305,20230307,09:00:01,0,4206.0,340295,1.432484e+10,4207.0,4209.0,4210.0,...,1.797693e+308,42095.347684,2.0,0.778151,0.602060,1.707570,1.518514,1.255273,1.255273,-0.012945
1482,rb2305,20230307,09:00:01,500,4207.0,340591,1.433729e+10,4210.0,4211.0,4212.0,...,1.797693e+308,42095.323482,1.0,1.255273,2.352183,2.075547,1.908485,1.518514,1.518514,-0.010787
1957,rb2305,20230307,09:00:02,0,4206.0,340773,1.434494e+10,4206.0,4207.0,4208.0,...,1.797693e+308,42095.308607,-1.0,1.301030,2.408240,1.301030,1.681241,1.826075,1.826075,-0.012945
2456,rb2305,20230307,09:00:02,500,4205.0,340909,1.435066e+10,4206.0,4207.0,4208.0,...,1.797693e+308,42095.292321,-1.0,1.113943,2.320146,1.204120,0.602060,1.875061,1.875061,-0.015102


In [3]:
#model = PPO("MlpPolicy", env, learning_rate=3e-3, tensorboard_log="tb_log", verbose=1, device="cuda") # cuda or cpuPPO
model = PPO("MlpPolicy", env, learning_rate=3e-4, tensorboard_log="tb_log", verbose=0, device="cuda") # cuda or cpu
model.learn(total_timesteps= int(3e5)) # this RETURN a trained model


In [4]:
%tensorboard --logdir tb_log

UsageError: Line magic function `%tensorboard` not found.


In [5]:
model.save("future_1")

In [7]:
obs, info = env.reset()

#print(info)
count = 0
run_limit = 50000
while True:
    count = count + 1
    action, _ = model.predict(obs, deterministic=False)
    #action, _ = model.predict(obs)
    #action = env.action_space.sample()
    #print(type(action))
    #print(action)
    #print(action.item(0))
    #break
    obs, reward, done, trunced, info = env.step(action)
    print(obs)
    if done or trunced:
        print("DONE---------")
        print(info)
        env.reset()
        break
    else:
        if info['ac'] != 2:
            print(info)
    
    if count > run_limit:
        break

#env.render_all()

[ 1.         -0.01078749  1.25527251 ...  2.00860017  2.
 -4.        ]
[-1.         -0.01294498  1.30103    ...  2.00860017  2.
 -9.        ]
[ -1.          -0.01510248   1.11394335 ...   2.01283722   2.
 -12.        ]
[  0.          -0.01510248   2.07554696 ...   2.01283722   2.
 -13.        ]
[  0.          -0.01510248   1.30103    ...   2.01283722   2.
 -14.        ]
[ -1.          -0.01725998   1.         ...   2.04139269   2.
 -14.        ]
[  0.          -0.01725998   1.94448267 ...   2.04921802   2.
 -14.        ]
[ -1.          -0.01941748   1.96378783 ...   2.13033377   2.
 -15.        ]
[  1.          -0.01725998   0.         ...   2.1931246    2.
 -15.        ]
[  0.          -0.01725998   1.95424251 ...   2.16731733   2.
 -15.        ]
[  1.          -0.01510248   2.18184359 ...   2.16731733   2.
 -15.        ]
[ -1.          -0.01725998   1.59106461 ...   2.16731733   2.
 -15.        ]
[ -1.          -0.01941748   1.43136376 ...   2.16731733   2.
 -15.        ]
[  1.      